In [1]:
import sys
from pathlib import Path

PROJECT_ROOT = Path.cwd().resolve().parents[0]
sys.path.insert(0, str(PROJECT_ROOT))

from sqlalchemy import text
from src.db import engine


In [2]:
sql = """
CREATE SCHEMA IF NOT EXISTS derived;

DROP MATERIALIZED VIEW IF EXISTS derived.mv_aki_icu_first_cohort;

CREATE MATERIALIZED VIEW derived.mv_aki_icu_first_cohort AS
WITH aki_hadm AS (
  SELECT DISTINCT subject_id, hadm_id
  FROM diagnoses_icd
  WHERE icd9_code LIKE '584%'
),
first_icu AS (
  SELECT DISTINCT ON (subject_id, hadm_id)
    subject_id,
    hadm_id,
    icustay_id,
    intime,
    outtime
  FROM icustays
  ORDER BY subject_id, hadm_id, intime
)
SELECT
  i.subject_id,
  i.hadm_id,
  i.icustay_id,
  i.intime,
  i.outtime,
  p.gender,
  p.dob,
  adm.admittime,
  adm.dischtime,
  adm.deathtime,
  adm.ethnicity,
  EXTRACT(YEAR FROM age(adm.admittime, p.dob)) AS age,
  CASE WHEN adm.deathtime IS NOT NULL THEN 1 ELSE 0 END AS hospital_mortality
FROM first_icu i
JOIN aki_hadm a
  ON a.subject_id = i.subject_id AND a.hadm_id = i.hadm_id
JOIN patients p
  ON p.subject_id = i.subject_id
JOIN admissions adm
  ON adm.subject_id = i.subject_id AND adm.hadm_id = i.hadm_id
WITH NO DATA;

REFRESH MATERIALIZED VIEW derived.mv_aki_icu_first_cohort;

"""

with engine.begin() as conn:
    conn.execute(text(sql))
    # FALSCH: conn.execute(text("REFRESH MATERIALIZED VIEW derived.mv_aki_dx_cohort;"))
    # RICHTIG:
    conn.execute(text("REFRESH MATERIALIZED VIEW derived.mv_aki_icu_first_cohort;"))


In [3]:
index_sql = """
CREATE UNIQUE INDEX IF NOT EXISTS idx_mv_aki_icu_first_cohort_icustay
ON derived.mv_aki_icu_first_cohort (icustay_id);

CREATE INDEX IF NOT EXISTS idx_mv_aki_icu_first_cohort_hadm
ON derived.mv_aki_icu_first_cohort (hadm_id);

CREATE INDEX IF NOT EXISTS idx_mv_aki_icu_first_cohort_subject
ON derived.mv_aki_icu_first_cohort (subject_id);
"""
with engine.begin() as conn:
    conn.execute(text(index_sql))


In [ ]:
from pathlib import Path
from sqlalchemy import text
from src.db import engine

sql_path = Path.cwd().resolve().parents[0] / "sql" / "concepts_postgres" / "severityscores" / "sofa.sql"
sofa_sql = sql_path.read_text(encoding="utf-8")
#C:\Users\Simon\Documents\Studium\5_Semester\DataTypes\ehr-project\sql\concepts_postgres\severityscores\sofa.sql

# Optional aber oft nötig:
# Viele mimic-code Skripte schreiben in ein Schema wie mimiciii_derived o.ä.
# Falls das bei dir so ist, ersetze es einmalig auf dein Schema "derived":
sofa_sql = sofa_sql.replace("mimiciii_derived", "derived")

with engine.begin() as conn:
    conn.execute(text("CREATE SCHEMA IF NOT EXISTS derived;"))
    conn.execute(text(sofa_sql))


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\tobra\\biomedical\\mimiciii-BioMedicalProject\\sql\\mimic\\sofa.sql'

In [ ]:
from src.db import q

q("""
SELECT
  COUNT(*) AS n_hadm,
  COUNT(DISTINCT subject_id) AS n_subject
FROM derived.mv_aki_dx_cohort
""")


In [ ]:
q("""
SELECT indexname, indexdef
FROM pg_indexes
WHERE tablename = 'mv_aki_icu_first_cohort';
""")
